### Load the data

In [393]:
# Useful starting lines
%matplotlib inline
import numpy as np
import datetime
import matplotlib.pyplot as plt

from helpers import *
from implementations import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
y, tx, ids = load_csv_data("train.csv", sub_sample=False)


In [415]:
y, tx, ids = load_csv_data("train.csv", sub_sample=False)

# change outlier values to the mean without them
mean = []
for i in range(tx.shape[1]):
    tx[:, i][tx[:, i] == -999] = np.mean(tx[:, i][tx[:, i] != -999])
    mean.append(np.mean(tx[:, i][tx[:, i] != -999]))
# transform the categorical features into one-hot features
#tx_ = np.delete(tx, 22, axis = 1)
#tx = np.hstack((tx,tx_**2,tx_**3,np.sin(tx_),np.cos(tx_),np.sin(2*tx_),np.cos(2*tx_),np.sin(tx_)**2,np.cos(tx_)**2,np.sin(0.5*tx_),np.cos(0.5*tx_),
#    (tx[:, 22] == 0).astype(np.float64)[:, np.newaxis],
#    (tx[:, 22] == 1).astype(np.float64)[:, np.newaxis],
#    (tx[:, 22] == 2).astype(np.float64)[:, np.newaxis],
#    (tx[:, 22] == 3).astype(np.float64)[:, np.newaxis]))
ind = np.where(tx[:, 22] > 1)
tx_ = np.delete(tx[tx[:, 22] > 1], 22, axis = 1)
tx__ = np.hstack([tx[tx[:, 22] > 1],tx_[:,0].reshape((tx_.shape[0],1))*tx_,tx_[:,1].reshape((tx_.shape[0],1))*tx_,tx_[:,2].reshape((tx_.shape[0],1))*tx_,tx_[:,3].reshape((tx_.shape[0],1))*tx_,tx_[:,4].reshape((tx_.shape[0],1))*tx_,tx_[:,5].reshape((tx_.shape[0],1))*tx_,tx_[:,6].reshape((tx_.shape[0],1))*tx_,
                tx_[:,7].reshape((tx_.shape[0],1))*tx_,np.sin(tx_),tx_[:,8].reshape((tx_.shape[0],1))*tx_,
                tx_[:,9].reshape((tx_.shape[0],1))*tx_,tx_[:,10].reshape((tx_.shape[0],1))*tx_,
                tx_[:,11].reshape((tx_.shape[0],1))*tx_,tx_[:,12].reshape((tx_.shape[0],1))*tx_,tx_[:,13].reshape((tx_.shape[0],1))*tx_,
                tx_[:,14].reshape((tx_.shape[0],1))*tx_,tx_[:,15].reshape((tx_.shape[0],1))*tx_,tx_[:,16].reshape((tx_.shape[0],1))*tx_,
                tx_[:,17].reshape((tx_.shape[0],1))*tx_,tx_[:,18].reshape((tx_.shape[0],1))*tx_,tx_[:,19].reshape((tx_.shape[0],1))*tx_,
                np.cos(tx_),np.sin(2*tx_),np.cos(2*tx_)])
tx = np.hstack([tx_])
tx = np.delete(tx, 22, axis = 1)

# standardization
tx_mean = np.mean(tx, axis=0)
tx_std = np.std(tx, axis=0)
tx = (tx - tx_mean) / tx_std


In [414]:
#y.shape, tx.shape
np.mean(tx, axis = 0).shape


(725,)

### Gradient Descent

In [236]:
# Define the parameters of the algorithm.
max_iters = 10
gamma = 0.01

# Initialization
initial_w = np.zeros(tx.shape[1])

# Start gradient descent.
start_time = datetime.datetime.now()
gd_ws, gd_losses= least_squares_GD(y, tx, initial_w, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time = {t:.3f} seconds".format(t=exection_time))
print("Gradient Descent: loss = {t}".format(t=gd_losses))

Gradient Descent: execution time = 0.417 seconds
Gradient Descent: loss = 0.4242524809016546


In [265]:
y_pred = tx @ gd_ws
y_pred[y_pred > 0] = 1
y_pred[y_pred < 0] = -1
print('Gradient descent accuracy:', np.mean(y_pred == y))

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 178 is different from 207)

### Stochastic Gradient Descent

In [349]:
# Define the parameters of the algorithm.
max_iters = 10
gamma = 0.0005

# Initialization
initial_w = sgd_ws#np.zeros(tx.shape[1])

# Start gradient descent.
start_time = datetime.datetime.now()
sgd_ws, sgd_losses= least_squares_SGD(y[ind], tx, initial_w, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Stochastic Gradient Descent: execution time = {t:.3f} seconds".format(t=exection_time))
print("Stochastic Gradient Descent: loss = {t}".format(t=sgd_losses))

Stochastic Gradient Descent: execution time = 10.892 seconds
Stochastic Gradient Descent: loss = 1101042270.8749208


In [348]:
y_pred = tx @ sgd_ws
y_pred[y_pred > 0] = 1
y_pred[y_pred < 0] = -1
print('Stochastic gradient descent accuracy:', np.mean(y_pred == y[ind]))

Stochastic gradient descent accuracy: 0.7947286437009774


### Least Squares

In [66]:
# Start least squares.
start_time = datetime.datetime.now()
ls_ws, ls_losses= least_squares(y, tx)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Least Squares: execution time = {t:.3f} seconds".format(t=exection_time))
print("Least Squares: loss = {t}".format(t=ls_losses))

Least Squares: execution time = 0.077 seconds
Least Squares: loss = 0.07619130801779375


In [67]:
y_pred = tx @ ls_ws
y_pred[y_pred > 0] = 1
y_pred[y_pred < 0] = -1
print('Least squares accuracy:', np.mean(y_pred == y))

Least squares accuracy: 0.657088


### Ridge Regression

In [224]:
# initialization
degree = 1
ratio = 0.8
seed = None
lambdas = np.logspace(-20, -15, 150)

# select lambda
lambda_ = hyperparameter_tuning_ridge_regression(tx, y, degree, ratio, seed, lambdas)

# Start ridge regression
start_time = datetime.datetime.now()
r_ws, r_losses = ridge_regression(y, tx, lambda_)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Ridge Regression: execution time = {t:.3f} seconds".format(t=exection_time))
print("Hyperparameter Selection: Lambda = {t}".format(t=lambda_))
print("Ridge Regression: loss = {t}".format(t=r_losses))

Ridge Regression: execution time = 0.270 seconds
Hyperparameter Selection: Lambda = 1e-15
Ridge Regression: loss = 0.29285213364144486


In [225]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold.
    return dimension: k-fold * interval
    """
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval] for k in range(k_fold)] 
    return np.array(k_indices)

In [226]:
# Cross Validation

# Initialization
seed = None
k_fold = 5
k_indices = build_k_indices(y, k_fold, seed)

train_acc = []
val_acc = []
for k in range(k_fold):
    val_y = y[k_indices[k, :]]
    val_x = tx[k_indices[k, :]]
    train_idx = np.vstack([k_indices[:k, :], k_indices[k+1:, :]]).flatten()
    train_y = y[train_idx]
    train_x = tx[train_idx]

    print('------', k, 'fold ------')
    w, loss = ridge_regression(train_y, train_x, lambda_)
    pred = train_x @ w
    pred[pred > 0] = 1
    pred[pred < 0] = -1
    train_acc.append(np.mean(pred == train_y))
    print('Train acc:', np.mean(pred == train_y))
    pred = val_x @ w
    pred[pred > 0] = 1
    pred[pred < 0] = -1
    val_acc.append(np.mean(pred == val_y))
    print('Validation acc:', np.mean(pred == val_y))

print('------ summary ------')
print('Average Train acc:', sum(train_acc)/len(train_acc))
print('Average Validation acc:', sum(val_acc)/len(val_acc))

------ 0 fold ------
Train acc: 0.800715
Validation acc: 0.8008
------ 1 fold ------
Train acc: 0.799585
Validation acc: 0.79806
------ 2 fold ------
Train acc: 0.799675
Validation acc: 0.79822
------ 3 fold ------
Train acc: 0.799045
Validation acc: 0.79866
------ 4 fold ------
Train acc: 0.799135
Validation acc: 0.79862
------ summary ------
Average Train acc: 0.799631
Average Validation acc: 0.798872


## Logistic Regression

In [11]:
# Define the parameters of the algorithm.
max_iters = 2
gamma = 0.0005


# Initialization
initial_w = np.zeros((tx.shape[1]))

# Start gradient descent.
start_time = datetime.datetime.now()
gd_ws, gd_losses= logistic_regression(y, tx, initial_w, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time = {t:.3f} seconds".format(t=exection_time))
print("Gradient Descent: loss = {t}".format(t=gd_losses))

Gradient Descent: execution time = 11.558 seconds
Gradient Descent: loss = inf


/Users/kieranvaudaux/Desktop/Machine-Learning-Project/implementations.py:150: RuntimeWarning: divide by zero encountered in log
  X_tx = tx@w


In [160]:
def predict_labels_logit(w, tx):
    y_pred = sigmoid(np.dot(tx, w))
    y_pred[np.where(y_pred <= 0.5)] = -1
    y_pred[np.where(y_pred > 0.5)] = 1
    return y_pred
# predictions
y_pred = predict_labels_logit(gd_ws, tx)

# accuracy
(y_pred == y).mean()

0.732784

## Regularized logistic regression

In [386]:
# Define the parameters of the algorithm.
max_iters = 3
gamma = 0.0001
lambda_ = 0.00001

# Initialization
initial_w = np.zeros((tx.shape[1]))

# Start gradient descent.
start_time = datetime.datetime.now()
gd_ws, gd_losses= reg_logistic_regression(y[ind], tx,lambda_, initial_w, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time = {t:.3f} seconds".format(t=exection_time))
print("Gradient Descent: loss = {t}".format(t=gd_losses))

Gradient Descent: execution time = 6.129 seconds
Gradient Descent: loss = 790556.3366674028


In [387]:
# predictions
y_pred = predict_labels_logit(gd_ws, tx)

# accuracy
(y_pred == y[ind]).mean()

0.7247701363329335

In [193]:
#reg_logit_loss(y,tx,lambda_,gd_ws)
#(sigmoid(tx@gd_ws)<0).sum()
#tx@gd_ws

In [213]:
x = np.genfromtxt("test.csv", delimiter=",", skip_header=1)
ids = x[:, 0].astype(np.int)
txx = x[:, 2:]
    
for i in range(txx.shape[1]):
    txx[:, i][txx[:, i] == -999] = mean[i]
# transform the categorical features into one-hot features
txx_= np.delete(txx, 22, axis = 1)
txx = np.hstack((txx,txx_**2,np.sin(txx_),np.cos(txx_),np.sin(2*txx_),np.cos(2*txx_),
        (txx[:, 22] == 0).astype(np.float64)[:, np.newaxis],
        (txx[:, 22] == 1).astype(np.float64)[:, np.newaxis],
        (txx[:, 22] == 2).astype(np.float64)[:, np.newaxis],
        (txx[:, 22] == 3).astype(np.float64)[:, np.newaxis]))
txx = np.delete(txx, 22, axis = 1)

    # standardization
#txx_mean = np.mean(txx, axis=0)
#txx_std = np.std(txx, axis=0)
txx[:, :-4] = (txx[:, :-4] - tx_mean[:-4]) / tx_std[:-4]
    

/var/folders/c4/wb13541d4n104gj2c8s0pdf40000gn/T/ipykernel_89830/608895844.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ids = x[:, 0].astype(np.int)


In [214]:
y_pred = predict_labels_logit(gd_ws, txx).astype(int)
y_pred[y_pred==0]=-1
print(np.hstack((ids.reshape(len(ids),1),y_pred.reshape(len(ids),1))).shape)

(568238, 2)


In [215]:
import pandas as pd 

d = pd.DataFrame(np.hstack((ids.reshape(len(ids),1),y_pred.reshape(len(ids),1))), columns = ["Id","Prediction"])
d.to_csv("submission_test_1.csv",index=False)